# Table of Contents
* [This exercise is a continuation of the EX02 example in the Introduction material for Python classes](#This-exercise-is-a-continuation-of-the-EX02-example-in-the-Introduction-material-for-Python-classes)
* [Here is the new PointND class](#Here-is-the-new-PointND-class)
* [And here is the ```MarkovChain``` class](#And-here-is-the-MarkovChain-class)
* [The problem](#The-problem)


# This exercise is a continuation of the EX02 example in the Introduction material for Python classes

In that example we set up a Point3D class to represent a point in 3-D space, and a Strategy class to help us simulate bacterial chemotaxis (the movement of a bacterium towards a favorable location in space by sensing gradients).

Let's modify those classes as follows:

 1. Change Point3D to represent a point in N-dimensional parameter space
 2. Convert the ```Strategy``` class to a ```MarkovChain``` class (See also [https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm)).  (Markov chain sampling can be used to measure parameter uncertainty)
 3. Use numpy throughout the classes to make it easy to have an N-dimensional space

# Here is the new PointND class

It takes initial points along with other arguments used for sampling the N-d space, including:
 
 * parameter mean
 * parameter covariance
 * minimum parameter bounds
 * maximum parameter bounds

In [ ]:
import numpy as np

class PointND(object):
    def __init__(self, coords, init_mean, init_cov, min_bound, max_bound):
        '''PointND represents a point in N-d space that can be updated based on stats.
        
        Inputs:
            coords is a numpy array representing initial coordinates in N-d space
            init_mean: is a mean array as long as coords for use in sampling new parameters
            init_cov: is a N x N covariance matrix for use in sampling new parameters
            min_bound: is the lower limit of parameter feasibility (array of shape (N, ))
            max_bound: is the upper limit parameter feasibility (array of shape (N,))
        '''
        self.coords = coords.copy()
        self.cov = init_cov
        self.mean = init_mean
        self.min_bound = min_bound
        self.max_bound = max_bound
        
    def _next_coords(self):
        '''Generates new parameters but parameters may be 
        out of bounds.'''
        return np.random.multivariate_normal(self.mean, self.cov)

    def next_coords(self):
        '''Calls the _next_coords method, ensures the new parameters
        are within min_bound and max_bound, and calls p_value_func
        for those parameters'''
        new_coords = self._next_coords()
        self.old_p_value = self.p_value
        new_coords[new_coords > self.max_bound] = self.max_bound[new_coords > self.max_bound]
        new_coords[new_coords < self.min_bound] = self.min_bound[new_coords < self.min_bound]
        self.coords = new_coords
        self.p_value_func()
        
    def p_value_func(self):
        '''When this function is called, it should define self.model and self.p_value'''
        raise NotImplementedError('Create this function in an inheriting class')

# And here is the ```MarkovChain``` class

To initialize it we give:

* ```p_value_func```, a function that returns a tuple of (p_value, model) for a given parameter set (see ```def p_value_func``` below).
* ```*args``` and  ```**kwargs```, which are mostly parameters passed to ```PointND```
* ```kwargs['alpha_accept']```, the probability of accepting a solution with a higher (worse) p value than the previous step. If the step's p value is better than the previous step's p value, then always accept.  This ```kwarg``` is not passed to ```PointND```

In [ ]:
class MarkovChain(PointND):
    default_alpha_accept = 0.3
    def __init__(self, p_value_func, *args, **kwargs):
        '''Initializes a Markov chain
        
        Inputs:
        
            p_value_func: a function called with self.coords as argument and returns 
            a tuple of (p_value, any_other_model_info_as_tuple)
            *args: arguments to PointND
            **kwargs: keyword arguments to PointND, as well as alpha_accept
                alpha_accept: determines the probability of acceptance of less likely 
                parameters, e.g. 0.3 means that 30 percent of the time, a less likely 
                parameter set is included in path
        '''
        if 'alpha_accept' in kwargs:
            self.alpha_accept = kwargs.pop('alpha_accept')
        else:
            self.alpha_accept = self.default_alpha_accept
        super(MarkovChain, self).__init__(*args, **kwargs)
        self._p_value_func = p_value_func
        self.p_value_func()
        self.old_p_value = self.p_value
                
    def p_value_func(self):
        '''Calls the user-given p_value_func to __init__ and stores output at 
        self.p_value, self.model
        '''
        if hasattr(self, 'p_value'):
            self.old_p_value = self.p_value
        self.p_value, self.model = self._p_value_func(self.coords)
         
    def keep_this_trial(self):
        '''Use this in a context where '''
        if self.p_value < self.old_p_value:
            return True
        return np.random.uniform(0, 1) < self.alpha_accept
    
    def make_path(self, *args, **kwargs):
        """This should initialize a list or numpy array that will be the Markov chain
        path.  Then it should step num_steps times and keep the trace of the Markov chain.
        """
        raise NotImplementedError('Write this function in inheriting class')

# The problem

* Make a class called ```MarkovChainPaths``` that inherits from ```MarkovChain```.
 1. Implement the ```make_path``` method in ```MarkovChainPaths```.  The ```make_path``` method should:
   * Provide some arguments for controlling the number of steps to take
   * Initialize numpy arrays for storing the model ```self.coords```, ```self.model``` and ```self.p_value``` attributes on each accepted step. Note that ```self.p_value``` is a scalar; ```self.model``` has a length of 4 in the example; and ```self.coords``` has the length of ```init_coords``` passed into ```__init__```
   * Probably use ```self.keep_this_trial()``` do decide whether to keep the new point in the path.  
 2. Extra challenge A: In your implementation of ```make_path```, have it also accept a ```burn_in``` integer argument that will have the effect not storing the first ```burn_in``` accepted solutions in the path.   This is a means of initializing the Markov chain.
 3. Extra challenge B: provide a way of updating the ```cov``` (coords' covariance) and ```mean``` (coords' mean) attributes based on accepted solutions along the path.  Updating those will change the generation of new ```coords```.  This is known as an adaptive Markov chain.
   * Probably the ```mean``` and ```cov``` would be updated periodically and you would have to check to see if the path meets some minimum length for redoing the statistics used for sampling.  
   * In the example below, the ```__init__``` takes an ```update_cov_mod``` keyword argument and stores it in ```self.update_cov_mod```.  Make ```update_cov_mod``` be an integer keyword argument that determines the frequency at which the covariance and mean are updated during the path evolution, e.g. ```if accepted_solutions % self.update_cov_mod == 0:```.  
   * Look at the documentation for ```np.cov``` and ```np.mean``` if you are unsure.

In [ ]:
class MarkovChainPaths(MarkovChain):
    def __init__(self, *args, **kwargs):
        '''MarkovChainPaths class to be updated.  Inherits from MarkovChain
        Inputs:
           *args: arguments passed to MarkovChain
           **kwargs: keyword arguments passed to MarkovChain, as well as:
               update_cov_mod: optional keyword that could be used to control
                   updating the covariance and mean used in sampling parameters.
        '''
        if 'update_cov_mod' in kwargs:
            self.update_cov_mod = kwargs.pop('update_cov_mod')
        else:
            self.update_cov_mod = None
        super(MarkovChainPaths, self).__init__(*args, **kwargs)
            
    def __len__(self):
        '''Requires _idx to be updated as arrays are filled out'''
        return getattr(self, '_idx', 0)
    
    def _append_to_path(self):
        '''Private method to advance the _idx used by __len__
        and store the current step's output in path, path_coords, model_fit_path arrays'''
        idx = len(self)
        self.path[idx] = self.p_value
        self.path_coords[idx, :] = self.coords
        self.model_fit_path[idx, :] = self.model
        self._idx = idx + 1
        
    def update_cov_mean(self):
        '''Updates the covariance and mean used in generation of new parameters'''
        if self.update_cov_mod is not None:              # if parameter was given
            if self.accepted % self.update_cov_mod == 0: # if we are at update step
                if len(self) >= self.update_cov_mod:     # if not the zero-th step:
                    # make sure cov is N x N (N = num parameters)
                    self.cov = np.cov(self.path_coords[:len(self)].T) 
                    # make sure mean is N long
                    self.mean = np.mean(self.path_coords[:len(self)],axis=0)
                    
    def make_path(self, num_accepted=1000, burn_in=500):
        '''make_path makes a Markov chain path
        
        Inputs:
            num_accepted: Simulate a path until num_accepted parameter sets 
                          have been accepted
            burn_in: Do not accumulate the first burn_in number of parameter sets 
                     (initialization)
        '''
        siz = num_accepted - burn_in
        self.path = np.empty(siz)
        self.path_coords = np.empty((siz, self.coords.size))
        self.model_fit_path = np.empty((siz, len(self.model)))
        if burn_in == 0:
            self._append_to_path()
        self.accepted = len(self)
        while len(self) < siz:
            self.next_coords()
            if self.keep_this_trial():
                if self.accepted >= burn_in:
                    self._append_to_path()
                    self.update_cov_mean()
                self.accepted += 1

In [ ]:
import numpy as np
from scipy import stats

lenn = 101
t = np.linspace(0, 100.0, lenn)
a,b,c = 5.0, 1.0, 2.0
real_series = a * t ** 3 + b * t + c + np.random.uniform(-.2*b, .2 * b, lenn)

def p_value_func(coords):
    guess = coords[0] * t ** 3 + coords[1] * t + coords[2]
    slope, intercept, r_value, p_value, std_err = stats.linregress(real_series, guess)
    return p_value, (slope, intercept, r_value, std_err)



init_coords = np.array([4.0,1.5, 1.0])
init_covariance = [[5.0] * 3] * 3
min_bound = np.array([0., 0., 0.])
max_bound = np.array([10.0, 2.0, 4.0])
markov = MarkovChainPaths(p_value_func, init_coords, init_coords, init_covariance,
                         min_bound, max_bound,alpha_accept=0.3,
                         update_cov_mod=100)
# then call markov.make_path with your arguments:
markov.make_path(num_accepted=10000, burn_in=9500)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
for column in range(init_coords.size):
    fig = plt.figure()
    ax = fig.gca()
    ax.hist(markov.path_coords[:,column])